<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

In [2]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    
import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


In [3]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [20]:
Y = torch.repeat_interleave(torch.arange(1024).view(1,1024), repeats=64, dim=0)
X = torch.repeat_interleave(torch.ones(1024).view(1,1024).float(), repeats=64, dim=0)

In [21]:
from neuralproc.predefined.cnn import CNN,ConvBlock
from neuralproc.predefined.mlp import MLP

In [43]:
class ModelTE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn1 = torch.nn.Conv2d(1,32,kernel_size=5,padding=2)
        self.cnn = CNN(32,ConvBlock,n_blocks=1,kernel_size=5,Conv=torch.nn.Conv2d)
        self.mlp = MLP(32, 1, n_hidden_layers=5)
        
    def forward(self, x):
        batch_size = x.size(0)
        signal = x.view(batch_size,1,32,32)
        conf = torch.ones_like(signal)
        
        signal = self.cnn1(signal * conf)
        conf = self.cnn1(conf)
        
        #out = conf
        out = torch.relu(signal / torch.clamp(conf, min=1e-5))
        
        out = self.cnn(out).view(batch_size,1024,32)
        return self.mlp(out).view(batch_size,1024)
    
class ModelNoTE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = MLP(1024, 1024, hidden_size=1024, n_hidden_layers=5)
        
    def forward(self, x):
        x = self.mlp(x)
        return x

class CELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.ce = torch.nn.CrossEntropyLoss()
        
    def forward(self, x, y):
        return self.ce(x,y.view(-1))
    
class MSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self, x, y):
        return self.mse(x,y)

In [44]:
from skorch import NeuralNetClassifier
from skorch.callbacks import EpochScoring

In [45]:
t=NeuralNetClassifier(ModelTE, criterion=MSELoss, batch_size=64, train_split=None, max_epochs=500, lr=0.001, optimizer=torch.optim.Adam)
t.fit(X,Y.float())

  epoch    train_loss     dur
-------  ------------  ------
      1   349014.8438  0.1077
      2   349009.4062  0.1072
      3   349004.7812  0.1132
      4   348999.6562  0.1109
      5   348994.1250  0.1283
      6   348988.3125  0.1240
      7   348982.1250  0.1205
      8   348975.6562  0.1208
      9   348968.8125  0.1174
     10   348961.5625  0.1446
     11   348953.8125  0.1228
     12   348945.5312  0.1157
     13   348936.5312  0.1212
     14   348926.7812  0.1197
     15   348916.1250  0.1113
     16   348904.4062  0.1211
     17   348891.4688  0.1153
     18   348877.0000  0.1113
     19   348860.8750  0.1111
     20   348842.8438  0.1107
     21   348822.7500  0.1125
     22   348800.0312  0.1126
     23   348774.2500  0.1185
     24   348744.9062  0.1177
     25   348711.2812  0.1221
     26   348672.5938  0.1134
     27   348627.7188  0.1148
     28   348575.7188  0.1154
     29   348515.5000  0.1113
     30   348445.6875  0.1139
     31   348364.0938  0.1505
     32   

    218     1028.6470  0.1180
    219     1019.3512  0.1128
    220     1010.2870  0.1164
    221     1000.6165  0.1143
    222      991.5228  0.1150
    223      982.5729  0.1149
    224      973.8190  0.1142
    225      964.4050  0.1138
    226      956.0342  0.1146
    227      945.6125  0.1148
    228      937.4152  0.1159
    229      929.2355  0.1166
    230      921.0612  0.1194
    231      912.2197  0.1229
    232      904.1348  0.1109
    233      896.1378  0.1175
    234      888.2490  0.1127
    235      880.4722  0.1181
    236      879.6259  0.1111
    237      871.3763  0.1098
    238      863.4486  0.1122
    239      857.5439  0.1176
    240      849.5482  0.1117
    241      841.5575  0.1152
    242      841.1783  0.1191
    243      832.8992  0.1145
    244      824.6205  0.1129
    245      816.4940  0.1132
    246      808.5233  0.1102
    247      800.7816  0.1168
    248      793.2152  0.1137
    249      786.5362  0.1130
    250      779.7098  0.1503
    251   

    445      218.7099  0.1115
    446      217.8700  0.1129
    447      215.7741  0.1095
    448      213.7635  0.1091
    449      212.2171  0.1055
    450      210.7597  0.1051
    451      209.2216  0.1063
    452      207.5259  0.1087
    453      205.7140  0.1094
    454      203.9004  0.1091
    455      202.1952  0.1134
    456      200.6457  0.1112
    457      199.2235  0.1106
    458      197.8539  0.1086
    459      196.4640  0.1099
    460      195.0223  0.1087
    461      193.5509  0.1094
    462      192.1006  0.1070
    463      190.7162  0.1069
    464      189.4125  0.1204
    465      188.1696  0.1144
    466      186.9491  0.1091
    467      185.7155  0.1069
    468      184.4554  0.1083
    469      183.1804  0.1072
    470      181.9158  0.1092
    471      180.6833  0.1064
    472      180.0690  0.1064
    473      178.9082  0.1092
    474      177.7612  0.1077
    475      176.6116  0.1040
    476      175.4558  0.1037
    477      174.2992  0.1031
    478   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ModelTE(
    (cnn1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (cnn): CNN(
      (conv_blocks): ModuleList(
        (0): ConvBlock(
          (activation): ReLU()
          (conv): DepthSepConv(
            (depthwise): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=32)
            (pointwise): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (norm): Identity()
        )
      )
    )
    (mlp): MLP(
      (dropout): Identity()
      (activation): ReLU()
      (to_hidden): Linear(in_features=32, out_features=32, bias=True)
      (linears): ModuleList(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): Linear(in_features=32, out_features=32, bias=True)
        (2): Linear(in_features=32, out_features=32, bias=True)
        (3): Linear(in_features=32, out_features=32, bias=True)
      )
      (out): Linear(in_feature

In [46]:
t.module_(X)[0,:].round().int()

tensor([  13,   41,   23,  ..., 1027, 1009,  932], dtype=torch.int32)